In [62]:
import matplotlib.pyplot as plt
import networkx as nx
import networkx.algorithms.isomorphism as iso
import sympy
import random
import time
import itertools
import math
from IPython.display import clear_output

In [63]:
def graphlet_list(N):
    assert N > 0
    foo = 1
    loc_graphlet_list = {n: [] for n in range(1,N+1)}
    while True:
        G = nx.graph_atlas(foo)
        n = G.number_of_nodes()
        if n>N:
            break
        if nx.is_connected(G):
            loc_graphlet_list[n].append(G)
        foo += 1
    return loc_graphlet_list
    

def find_type_match(T):
    
    n = T.number_of_nodes()
    if n==1:
        return((0, {u: 0 for u in T.nodes()}))
    if n==2:
        return((0, {u: i for i,u in enumerate(T.nodes())}))
    if n==3:
        if T.number_of_edges()==2:
            u0 = next((node for node in T.nodes() if T.degree(node)==2))
            (u1,u2) = (node for node in T.neighbors(u0))
            return((0, {u0: 0, u1: 1, u2: 2}))
        if T.number_of_edges()==3:
            return((1,{u:i for i,u in enumerate(T.nodes())}))
    if n==4:
        e_num = T.number_of_edges()
        max_degree = max((T.degree(node) for node in T.nodes()))
        if e_num==3 and max_degree==3:
            u3 = next((node for node in T.nodes() if T.degree(node)==3))
            (u0,u1,u2) = (node for node in T.neighbors(u3))
            return((0, {u0:0, u1:1, u2:2, u3:3}))
        if e_num==3 and max_degree==2:
            (u0,u1) = (node for node in T.nodes() if T.degree(node)==2)
            u2 = next((node for node in T.neighbors(u1) if node!=u0))
            u3 = next((node for node in T.neighbors(u0) if node!=u1))
            return((1, {u0:0, u1:1, u2:2, u3:3}))
        if e_num==4 and max_degree==3:
            u3 = next((node for node in T.nodes() if T.degree(node)==3))
            (u1,u2) = (node for node in T.nodes() if T.degree(node)==2)
            u0 = next((node for node in T.nodes() if T.degree(node)==1))
            return((2, {u0:0, u1:1, u2:2, u3:3}))
        if e_num==4 and max_degree==2:
            u0 = next((node for node in T.nodes()))
            (u1,u3) = (node for node in T.neighbors(u0))
            u2 = next((node for node in T.neighbors(u1) if node!=u0))
            return((3, {u0:0, u1:1, u2:2, u3:3}))
        if e_num==5:
            (u0,u2) = (node for node in T.nodes() if T.degree(node)==3)
            (u1,u3) = (node for node in T.nodes() if T.degree(node)==2)
            return((4, {u0:0, u1:1, u2:2, u3:3}))
        if e_num==6:
            (u0,u1,u2,u3) = (node for node in T.nodes())
            return((5, {u0:0, u1:1, u2:2, u3:3}))
    # Improve matching procedure here for n>4.
    GM = next((i, iso.GraphMatcher(T,T_)) 
              for (i,T_) in enumerate(cached_graphlet_list[n]) 
              if iso.GraphMatcher(T,T_).is_isomorphic())
    assert GM[1].is_isomorphic()    
    return((GM[0],GM[1].mapping))

def find_type_prob(G,T_vert):
    k = len(T_vert)
    T_type, T_match = find_type_match(subgraph(G,T_vert)) 
    inv_match = {i: j for j,i in T_match.items()}
    degree_list = [G.degree(inv_match[i]) for i in range(k)]
    T_prob = cached_prob[T_type](*degree_list)*(cached_edge_number)**(-1)
    return (T_type, T_prob)

def find_type(T):
    n = T.number_of_nodes()
    if n==1:
        return 0
    if n==2:
        return 0
    if n==3:
        if T.number_of_edges()==2:
            return 0
        if T.number_of_edges()==3:
            return 1
    if n==4:
        e_num = T.number_of_edges()
        max_degree = max((T.degree(node) for node in T.nodes()))
        if e_num==3 and max_degree==3:
            return 0
        if e_num==3 and max_degree==2:
            return 1
        if e_num==4 and max_degree==3:
            return 2
        if e_num==4 and max_degree==2:
            return 3
        if e_num==5:
            return 4
        if e_num==6:
            return 5
    # Improve matching procedure here at least for n=4.
    GM = next((i 
              for (i,T_) in enumerate(cached_graphlet_list[n]) 
              if iso.GraphMatcher(T,T_).is_isomorphic()))
    return GM  

def prob_functions(N):
    assert N > 0
    x = {0: sympy.var('x_0')}
    y = {0: sympy.var('y_0')}
    prob = {1: {0: x[0]/2}}
    if N > 1:
        x[1] = sympy.var('x_1')        
        y[1] = sympy.var('y_1')
        prob[2] = {0: sympy.Integer(1)}
    for n in range(3, N+1):
        x[n-1] = sympy.var('x_{}'.format(n-1))
        y[n-1] = sympy.var('y_{}'.format(n-1))
        prob[n] = {}
        for T_ind, T in enumerate(cached_graphlet_list[n]):
            prob[n][T_ind] = 0
            for u in T.nodes():
                S = subgraph(T, T.nodes()-{u})
                if not nx.is_connected(S):
                    continue
                S_ind, S_match = find_type_match(S)
                S_prob = (prob[n-1][S_ind]
                          .subs({x[i]:y[i] for i in range(n-1)})
                          .subs({y[j]:x[i] for i,j in S_match.items()})
                         )
                S_deg = sum(x[i] for i in S.nodes()) - 2*S.number_of_edges()
                prob[n][T_ind] += S_prob * T.degree(u) / S_deg                 
    return prob[N]

def subgraph(G, nodes):
    list_nodes = list(nodes)
    T = nx.Graph()
    T.add_nodes_from(nodes)
    for i in range(len(nodes)):
        for j in range(i):
            if list_nodes[i] in G.neighbors(list_nodes[j]):
                T.add_edge(list_nodes[i],list_nodes[j])
    return T


In [64]:
k=4
x = [sympy.var('x_{}'.format(i)) for i in range(k)]
cached_graphlet_list = graphlet_list(k)
cached_prob = {ind: sympy.lambdify(x, func) 
                    for ind, func in prob_functions(k).items()
              }
#4s for N=5, 2m43s for N=6

In [80]:
def lift(G, vert, k):
    graphlet = set([vert])
    if k==1:
        return graphlet
    u = vert
    neig_list = []
    for n in range(2, k+1):
        neig_list = ([v for v in neig_list if v!=u] 
                     + [v for v in G.neighbors(u) if v not in graphlet])
        u = random.choice(neig_list)
        graphlet.add(u)
    return graphlet

def random_walk_nodes(G, v0, steps_num):
    curr_vert = v0
    for _ in range(steps_num):
        curr_vert = random.choice(list(G.neighbors(curr_vert)))
    return curr_vert

def load_graph(name, k=3):
    
    ground_truth = None
    G = None

    if name=='com-amazon':
        G = nx.read_edgelist(
            'Graphs/com-amazon.ungraph.txt',
            create_using = nx.Graph())
        
        if k==3:
            ground_truth = {0: 7750799, 
                            1: 667129}
        if k==4:
            ground_truth = {0: 124295537, 
                            1: 37383434, 
                            2: 13674662, 
                            3: 422515, 
                            4: 1874925, 
                            5: 275961}

    if name=='com-dblp':
        G = nx.read_edgelist(
            'Graphs/com-dblp.ungraph.txt',
            create_using = nx.Graph())
        if k==3:
            ground_truth = {0: 15107734, 
                            1: 2224385}
        if k==4:
            ground_truth = {0: 258570802, 
                            1: 252447350, 
                            2: 96615211, 
                            3: 203394, 
                            4: 4764685, 
                            5: 16713192}

    if name=='com-lj':
        G = nx.read_edgelist(
            'Graphs/com-lj.ungraph.txt',
            create_using = nx.Graph())
        if k==3:
            ground_truth = {0: 3722307805, 
                            1: 177820130}
        if k==4:
            ground_truth = {0: 1983908933796,
                            1: 542683013686,
                            2: 57662704306,
                            3: 2541452010,
                            4: 8190586835,
                            5: 521691844}

    if name=='com-youtube':
        G = nx.read_edgelist(
            'Graphs/com-youtube.ungraph.txt',
            create_using = nx.Graph())
        if k==3:
            ground_truth = {0: 1465313402, 
                            1: 3056386}
        if k==4:
            ground_truth = {0: 5730407268993,
                            1: 91488735459,
                            2: 12371157628,
                            3: 231979854,
                            4: 221833272,
                            5: 4986965}

    if name=='misc-net25':
        G = nx.read_edgelist(
            'Graphs/misc-net25.mtx',
            create_using = nx.Graph())
        for v in G.nodes():
            G.remove_edge(v,v)
        if k==3:
            ground_truth = {0: 12690840, 
                            1: 64090}
        if k==4:
            ground_truth = {0: 361490550,
                            1: 550792350,
                            2: 12554670,
                            3: 44915955,
                            4: 0,
                            5: 0}

    if name=='bio-celegansneural':
        G = nx.read_edgelist(
            'Graphs/bio-celegansneural.mtx',
            create_using = nx.Graph(), data=(('weight',float),))

    if name=='bio-yeast':
        G = nx.read_edgelist(
            'Graphs/bio-yeast.mtx',
            create_using = nx.Graph())
    
    if name=='bn-macaque-rhesus_brain_1':
        G = nx.read_edgelist(
            'Graphs/bn-macaque-rhesus_brain_1.edges',
            create_using = nx.Graph())
    
    if name=='bn-mouse_brain_1':
        G = nx.read_edgelist(
            'Graphs/bn-mouse_brain_1.edges',
            create_using = nx.Graph())
    
    if name=='ia-email-univ':
        G = nx.read_edgelist(
            'Graphs/ia-email-univ.mtx',
            create_using = nx.Graph())

    if name=='misc-polblogs':
        G = nx.read_edgelist(
            'Graphs/misc-polblogs.mtx',
            create_using = nx.Graph(), data=(('weight',float),))
        

    if name=='misc-as-caida':
        G = nx.read_edgelist(
            'Graphs/misc-as-caida.mtx',
            create_using = nx.Graph(), data=(('weight',float),)) 
        if k==3:
            ground_truth = {0: 59513652, 
                            1: 72730}
        if k==4:
            ground_truth = {0: 62565214368,
                            1: 2808802860,
                            2: 203097552,
                            3: 3774144,
                            4: 4084544,
                            5: 0}

    if name=='misc-fullb':
        G = nx.read_edgelist(
            'Graphs/misc-fullb.mtx',
            create_using = nx.Graph())
        for v in G.nodes():
            G.remove_edge(v,v)
        if k==3:
            ground_truth = {0: 162067420, 
                            1: 60212260}
        if k==4:
            ground_truth = {0: 1078734774,
                            1: 4837795036,
                            2: 2707584768,
                            3: 64898820,
                            4: 897215295,
                            5: 370980150}

    if name=='misc-neos3':
        G = nx.read_edgelist(
            'Graphs/misc-neos3.mtx',
            create_using = nx.Graph(), data=(('weight',float),))

        if k==3:
            ground_truth = {0: 207426691, 
                            1: 505603}
        if k==4:
            ground_truth = {0: 59618248397,
                            1: 11164704825,
                            2: 120388385,
                            3: 2047846,
                            4: 499122,
                            5: 0}

    if name=='misc-discogs_affiliation':
        G = nx.read_edgelist(
            'Graphs/misc-discogs_affiliation.edges',
            create_using = nx.Graph())
        if k==3:
            ground_truth = None
        if k==4:
            ground_truth = {0: 208345722513295,
                            1: 851118877585,
                            2: 58223406336,
                            3: 3008868833,
                            4: 439215089,
                            5: 654413}

    if name=='misc-amazon-ratings':
        G = nx.read_edgelist(
            'Graphs/misc-amazon-ratings.edges',
            create_using = nx.Graph())
        if k==3:
            ground_truth = {0: 699425719, 
                            1: 79638}
        if k==4:
            ground_truth = {0: 719668204837,
                            1: 40966346985,
                            2: 184396006,
                            3: 37045086,
                            4: 561566,
                            5: 671}

    if name=='misc-dbpedia-all':
        G = nx.read_edgelist(
            'Graphs/misc-dbpedia-all.edges',
            create_using = nx.Graph())
        if k==3:
            ground_truth = {0: 174250340949, 
                            1: 8329548}
        if k==4:
            ground_truth = {0: 19646604300441472,
                            1: 1652259549599,
                            2: 622928133900,
                            3: 15925209557,
                            4: 15630164176,
                            5: 4609834}
            
    if G is None:
        raise KeyError

    return {'graph': G, 'ground_truth': ground_truth}

def lift_mixing_variance(G, k, steps_num=1000, burn_in_limit=20):
    v = random_walk_nodes(G,random.choice(list(G.nodes())),100)
    graphlet_num = len(cached_graphlet_list[k])
    exp_counter = {i:0 for i in range(graphlet_num)}
    type_counter = {i:0 for i in range(graphlet_num)}
    var_counter = {i:0 for i in range(graphlet_num)}
    pair_counter = {i: 
                    {burn_in: 0 
                     for burn_in in range(0,burn_in_limit)}
                    for i in range(graphlet_num)} 
    corr_counter = {i: 
                    {burn_in: 0 
                     for burn_in in range(0,burn_in_limit)}
                    for i in range(graphlet_num)} 
    memory = [None for _ in range(burn_in_limit)]
    for _ in range(steps_num):
        T = lift(G, v, k)
        v = random_walk_nodes(G, v, 1)
        T_type, T_prob = find_type_prob(G, T)
        type_counter[T_type] += 1
        exp_counter[T_type] += (T_prob)**(-1)        
        var_counter[T_type] += (T_prob)**(-2)
        ind = 0
        while ind < burn_in_limit and memory[ind] is not None:
            S_type, S_prob = memory[ind]
            if T_type==S_type:
                pair_counter[T_type][ind] += 1
                corr_counter[T_type][ind] += (T_prob*S_prob)**(-1)
            ind+=1
        memory = [(T_type, T_prob)] + memory[:-1]

    beta_coeff = {i: [abs(corr_counter[i][burn_in]
                          *(steps_num-burn_in)
                          *exp_counter[i]**(-2) - 1)
                      for burn_in in range(burn_in_limit)] 
                  for i in range(graphlet_num) 
                  if type_counter[i]!=0}

    variance = {i: (var_counter[i]*steps_num
                    *exp_counter[i]**(-2))
                for i in range(graphlet_num) 
                if exp_counter[i]!=0}

    for i in range(graphlet_num):
        if exp_counter[i]==0:
            continue
        print ("Graphlet ID{}".format(i))
        print("Expectation")
        print(exp_counter[i]*steps_num**(-1))
        print("Variance")
        print(variance[i])
        print("Count")
        print(type_counter[i])
        print(" ")
        
    for i in range(graphlet_num):
        if exp_counter[i]==0:
            continue
        print ("Graphlet ID{}".format(i))
        for burn_in in range(burn_in_limit):
            print(beta_coeff[i][burn_in])
        
    return (beta_coeff, variance)

def lift_count(G, k, steps_num=1000, burn_in=3):
    v = random_walk_nodes(G,random.choice(list(G.nodes())),100)
    graphlet_num = len(cached_graphlet_list[k])
    graphlet_counter = {i:0 for i in range(graphlet_num)}
    for _ in range(steps_num): 
        T = lift(G, v, k)
        T_type, T_prob = find_type_prob(G, T)        
        graphlet_counter[T_type] += (T_prob)**(-1)   
        v = random_walk_nodes(G,v,burn_in)     
    graphlet_counter = {i: var*steps_num**(-1) 
                        for i,var in graphlet_counter.items()}
    return graphlet_counter


In [66]:
G = load_graph('misc-neos3',4)['graph']
cached_edge_number = G.number_of_edges()
cached_vert_number = G.number_of_nodes()
print(cached_vert_number, cached_edge_number)

(518832, 2055024)


In [84]:
lift_mixing_variance(G, 4, steps_num=10**6, burn_in_limit=10)

Graphlet ID0
Expectation
58606960017.7
Variance
206.519998481
Count
718040
 
Graphlet ID1
Expectation
11190268146.7
Variance
240.304012865
Count
190814
 
Graphlet ID2
Expectation
119777833.809
Variance
13.7703631964
Count
73318
 
Graphlet ID3
Expectation
2066865.05247
Variance
22145.6705004
Count
75
 
Graphlet ID4
Expectation
501899.718799
Variance
56.3285080459
Count
17753
 
Graphlet ID0
0.265685096312
0.212897418754
0.263361212863
0.0154475954118
0.196969413367
0.0250902166712
0.0226820876142
0.246178493427
0.103557525603
0.123510156448
Graphlet ID1
0.12962036152
0.00897610455879
0.213560297013
0.228349753072
0.210846285215
0.134047966396
0.65477566916
0.142681427576
0.351493567132
0.00764623332305
Graphlet ID2
0.210601637706
0.133551573533
0.0713499696212
0.0475790255758
0.0463159368799
0.0199010897985
0.00354194397776
0.0172044736756
0.00478481209928
0.00597919289254
Graphlet ID3
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Graphlet ID4
0.381275423645
0.193011793406
0.095725142824
0.022

({0: [0.26568509631197057,
   0.21289741875446488,
   0.2633612128629278,
   0.01544759541184848,
   0.19696941336730345,
   0.02509021667120681,
   0.022682087614216417,
   0.2461784934273954,
   0.10355752560314357,
   0.12351015644793861],
  1: [0.1296203615196535,
   0.008976104558785947,
   0.21356029701263046,
   0.2283497530717017,
   0.2108462852147691,
   0.1340479663960754,
   0.654775669159918,
   0.1426814275763454,
   0.35149356713156377,
   0.007646233323051366],
  2: [0.21060163770644602,
   0.13355157353309832,
   0.07134996962118023,
   0.04757902557578908,
   0.046315936879892994,
   0.019901089798528115,
   0.003541943977764195,
   0.017204473675613396,
   0.004784812099283053,
   0.005979192892539675],
  3: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
  4: [0.3812754236445308,
   0.193011793406499,
   0.09572514282396738,
   0.022754026919510895,
   0.010056752249800627,
   0.0005428393717727964,
   0.06082618900465986,
   0.008982332300193407,
   0.062915330

In [ ]:
#Garbage code
def liftSRW(G, k, time_limit=None, query_limit=None, epoch_num=1, time_step=10, 
            output_form='count', ground_truth=None, steps_mult=None, steps_num=None):

    assert (time_limit is None) != (query_limit is None)
    assert (steps_mult is None) != (steps_num is None)
    
    if steps_mult is not None:
        steps_between_lifts = int(steps_mult*math.log(G.number_of_nodes()))
    else:
        steps_between_lifts = steps_num
    norm_error = 0
    
    for epoch in range(epoch_num):
        #print('Starting epoch {}'.format(epoch+1))
        type_counter = {i:0 for i in range(len(cached_graphlet_list[k]))}
        t0 = time.time()
        lift_count = 0
        query_count = 0
        time_iter_count = 1
        stop_condition = False
        v = random.choice(list(G.nodes()))
        flag = 0

        while not stop_condition:
            v = random_walk_nodes(G, v, steps_between_lifts)
            T = lift(G, v, k)
            T_type, T_match = find_type_match(subgraph(G,T))
            inv_match = {i: j for j,i in T_match.items()}
            degree_list = [G.degree(inv_match[i]) for i in range(k)]
            T_prob = cached_prob[T_type](*degree_list)
            type_counter[T_type] += (T_prob)**(-1)
            lift_count += 1
            curr_time = time.time()
            type_list = []
            
            if curr_time - t0 > time_iter_count*time_step:
                if output_form=='count':
                    print("Time is {} Type counter is {}"
                          .format(int(curr_time-t0), 
                                  scale(type_counter, 
                                        G.number_of_edges()*lift_count**(-1))))
                if output_form=='ratio':
                    print("Time is {} Type ratios are {}"
                          .format(int(curr_time-t0), 
                                  normalize(type_counter)))
                print("Time is {} NMSE is {}"
                      .format(int(curr_time-t0), 
                              NMSE(type_counter, ground_truth)))
                print("Number of graphlets sampled is {}".format(lift_count))
                time_iter_count += 1
                
            if time_limit is not None:
                stop_condition = (time.time()-t0 > time_limit)
            if query_limit is not None:
                query_count += steps_between_lifts + k - 1
                stop_condition = (query_count > query_limit)

        if ground_truth is not None:
            error = NMSE(type_counter, ground_truth)
            #print("NMSE error is {}".format(error))
            norm_error += error
        #print("")
    print("Number of lifts is {}".format(lift_count))
    norm_error = norm_error*(epoch_num)**(-1)
    return {'count': scale(type_counter, G.number_of_edges()*lift_count**(-1)), 
            'NMSE': norm_error}

def brute_force(G, N=3):
    type_counter = {i:0 for i in range(len(cached_graphlet_list[N]))}
    if N==3:
        percent_count = 0
        counter = 0
        for u,v in G.edges():
            for w in set(G.neighbors(u))-{v}:
                if w in G.neighbors(v): 
                    type_counter[1] += 1
                else:
                    type_counter[0] += 1
            for w in set(G.neighbors(v))-{u}:
                if w in G.neighbors(u): 
                    type_counter[1] += 1
                else:
                    type_counter[0] += 1
            counter += 1
            if counter > percent_count*0.00001*G.number_of_edges():
                clear_output()
                #print("{}% complete".format(percent_count*0.001))
                percent_count += 1
        type_counter[0] = type_counter[0]/2
        type_counter[1] = type_counter[1]/6
        
    if N==4:
        for u,v in G.edges():
            neigh = set(G.neighbors(u)).union(set(G.neighbors(v)))-{u,v}
            for w,z in itertools.combinations(neigh, 2):
                T = subgraph(G, {u,v,w,z})
                type_counter[find_type(T)] += 1
        type_counter[0] = type_counter[0]/3
        type_counter[1] = type_counter[1]
        type_counter[2] = type_counter[2]/3
        type_counter[3] = type_counter[3]/4
        type_counter[4] = type_counter[4]/5
        type_counter[5] = type_counter[5]/6
    return type_counter

def NMSE(dict_hat, dict_true):
    norm_dict_hat = normalize(dict_hat)
    norm_dict_true = normalize(dict_true)
    return sum(((norm_dict_hat[i]*freq**(-1) - 1)**2
                for i, freq in norm_dict_true.items() if freq != 0))

def normalize(dict_hat):
    total_count = sum((val for i,val in dict_hat.items()))
    return {i: val*(total_count)**(-1) for i,val in dict_hat.items()}

def scale(dict_hat, scalar):
    return {i: int(val*scalar) for i,val in dict_hat.items()}

def lift_variance(G, k, steps_num=1000, burn_in=3):
    v = random_walk_nodes(G,random.choice(list(G.nodes())),100)
    graphlet_num = len(cached_graphlet_list[k])
    variance_counter = {i:0 for i in range(graphlet_num)}
    expectation_counter = {i:0 for i in range(graphlet_num)}
    for _ in range(steps_num):
        T = lift(G, v, k)
        T_type, T_prob = find_type_prob(G,T)
        variance_counter[T_type] += (T_prob)**(-2)
        expectation_counter[T_type] += (T_prob)**(-1)
        v = random_walk_nodes(G,v,burn_in)
    norm_variance = {i: (variance_counter[i]**(0.5)
                         * steps_num**(-0.5))
                     for i in range(graphlet_num)}
    norm_expectation = {i: (expectation_counter[i]
                           * steps_num**(-1))
                     for i in range(graphlet_num)}
    return {'variance': norm_variance, 
            'expectation': norm_expectation}
